### This notebook uses a breathing actuation sequence that computes triggers inflation deflation based upon an average inhale/exhale duration and a scale factor

In [1]:
from pythonosc import dispatcher, osc_server
from pythonosc.udp_client import SimpleUDPClient
import biofeatures
import time
import threading
import numpy as np

In [2]:
def data_handler(unused_addr, args, data1, data2, data3, data4, data5, data6): #BITalino ServerBIT format
    global ecg_data, resp_data
    global osc_client
        
    B = args[0]
    HR = args[1]
    
    # BITalino ServerBIT
    ecg_data.append(float(np.random.rand(1)))
    resp_data.append(float(data6))
        
    if B.is_warmed_up:
        B.set_data(resp_data[-B.buffer_length:])
    
    if HR.is_warmed_up:
        HR.set_data(ecg_data[-HR.buffer_length:])


In [3]:
def warmup(B, resp_data, HR, ecg_data):
    """Function to launch once the system has warmed up.
    Sets the  data initially and launches a recursive update of features.
    Parameters
    ----------
    B: object containing breathing data and features
    resp_data: respiration signal data to set
    HR: object containing ECG data and features
    ecg_data: ecg signal data to set
    """
    
    if not B.is_warmed_up:
        print("Breathing WARMUP")
        B.set_data(resp_data[-B.buffer_length:])
        B.is_warmed_up = True
        B.update_loop()
        
    if not HR.is_warmed_up:
        print("ECG WARMUP")
        HR.set_data(ecg_data[-HR.buffer_length:])
        HR.is_warmed_up = True
        HR.update_loop()
        
    breathing_factor = 1.2
    
    timer_actuation1 = threading.Timer(0.1, breathing_intervals, ["1", 6, 6, True] )
    timer_actuation1.start()
    
    timer_actuation2 = threading.Timer(0.1, breathing_intervals, ["2", 3, 3, True] )
    #timer_actuation2 = threading.Timer(0.1, pulsating, ["2", True] )
    timer_actuation2.start()

In [4]:
def warmup_inflate(act_id, duration):
    global osc_client
    
    osc_client.send_message("/actuator/" + act_id + "/inflate", 100.0)
    time.sleep(duration)
    osc_client.send_message("/actuator/" + act_id + "/inflate", 0.0)


In [5]:
def deflate(act_id, duration):
    global osc_client
    
    osc_client.send_message("/actuator/" + act_id + "/inflate", -100.0)
    time.sleep(duration)
    osc_client.send_message("/actuator/" + act_id + "/inflate", 0.0)


### Different actuation sequences

In [6]:
# computes inhale and exhale interval duration as average duration of breath * breathing_factor 

def breathing_biofeedback(act_id, B, breathing_factor, inflate):
    global osc_client, actuation_flag
    
    if not actuation_flag:
        return
    
    if inflate:
        print("inhale: ", B.features['avg_inhale'])
        osc_client.send_message("/actuator/" + act_id + "/inflate", 100.0)
        timer_exhale = threading.Timer(B.features['avg_inhale'] * breathing_factor, breathing_biofeedback, [act_id, B, breathing_factor, False])
        timer_exhale.start()

    else:
        print("exhale: ", B.features['avg_inhale'])
        osc_client.send_message("/actuator/" + act_id + "/inflate", -100.0)
        timer_inhale = threading.Timer(B.features['avg_exhale'] * breathing_factor, breathing_biofeedback, [act_id, B, breathing_factor, True])
        timer_inhale.start()

In [13]:
# specify inhale and exhale interval duration in seconds

def breathing_intervals(act_id, inhale_duration, exhale_duration, inflate):
    global osc_client, actuation_flag
    
    if not actuation_flag:
        return
    
    if inflate:
        print("inhale: ", inhale_duration)
        osc_client.send_message("/actuator/" + act_id + "/inflate", 80.0)
        timer_exhale = threading.Timer(inhale_duration, breathing_intervals, [act_id, inhale_duration, exhale_duration, False])
        timer_exhale.start()

    else:
        print("exhale: ", exhale_duration)
        osc_client.send_message("/actuator/" + act_id + "/inflate", -80.0)
        timer_inhale = threading.Timer(exhale_duration, breathing_intervals, [act_id, inhale_duration, exhale_duration, True])
        timer_inhale.start()

In [8]:
def pulsating(act_id,inflate):
    global osc_client, actuation_flag
    
    if not actuation_flag:
        return
    
    if inflate:
        osc_client.send_message("/actuator/" + act_id + "/inflate", 50.0)
        timer_deflate = threading.Timer(3, pulsating, [act_id, False])
        timer_deflate.start()

    else:
        osc_client.send_message("/actuator/" + act_id + "/inflate", -50.0)
        timer_inflate = threading.Timer(3, pulsating, [act_id, True])
        timer_inflate.start()

In [ ]:
from random import uniform

def random_inflation(act_id, lower, upper, inflate):
    global osc_client, actuation_flag
    
    if not actuation_flag:
        return
    
    int_length = uniform(lower,upper)
    
    if inflate:
        osc_client.send_message("/actuator/" + act_id + "/inflate", 100.0)
        timer_deflate = threading.Timer(int_length, random_inflation, [act_id, lower, upper, False])
        timer_deflate.start()

    else:
        osc_client.send_message("/actuator/" + act_id + "/inflate", -100.0)
        timer_inflate = threading.Timer(int_length, random_inflation, [act_id, lower, upper, True])
        timer_inflate.start()

In [ ]:
def async_intervals(act_first, act_snd, inhale_dur, exhale_dur, inflate_first):
    global osc_client, actuation_flag
    
    if not actuation_flag:
        return
    
    if inflate:
        print("inhale: ", inhale_duration)
        osc_client.send_message("/actuator/" + act_first + "/inflate", 100.0)
        osc_client.send_message("/actuator/" + act_snd + "/inflate", -100.0)
        timer_exhale = threading.Timer(inhale_dur, async_intervals, [act_first, act_snd, inhale_dur, exhale_dur, False])
        timer_exhale.start()

    else:
        print("exhale: ", exhale_duration)
        osc_client.send_message("/actuator/" + act_first + "/inflate", -100.0)
        osc_client.send_message("/actuator/" + act_snd + "/inflate", 100.0)
        timer_exhale = threading.Timer(exhale_dur, async_intervals, [act_first, act_snd, inhale_dur, exhale_dur, True])
        timer_inhale.start()

In [ ]:
def async_breathing(act_first, act_snd, B, breathing_factor, inflate):
    global osc_client, actuation_flag
    
    if not actuation_flag:
        return
    
    if inflate:
        print("inhale: ", B.features['avg_inhale'])
        osc_client.send_message("/actuator/" + act_first + "/inflate", 100.0)
        osc_client.send_message("/actuator/" + act_snd + "/inflate", -100.0)
        timer_exhale = threading.Timer(B.features['avg_inhale'] * breathing_factor, async_breathing, [act_first, act_snd, B, breathing_factor, False])
        timer_exhale.start()

    else:
        print("exhale: ", B.features['avg_exhale'])
        osc_client.send_message("/actuator/" + act_first + "/inflate", -100.0)
        osc_client.send_message("/actuator/" + act_snd + "/inflate", 100.0)
        timer_inhale = threading.Timer(B.features['avg_exhale'] * breathing_factor, async_breathing, [act_first, act_snd, B, breathing_factor, True])
        timer_inhale.start()

In [9]:
# computes degree of inflation/deflation based on ECG features

# TODO: how to compute the inflation/deflation value?

def heartrate_actuation(act_id, HR, timer_interval):
    global osc_client, actuation_flag
    
    if not actuation_flag:
        return
    
    current_trend = HR.current_trends["hr_mean"]
    #current_trend = HR.current_trends["rmssd"]
    #current_trend = HR.current_trends["LF/HF ratio"]
    
    current_feature = HR.features[-1]["hr_mean"]
    #current_feature = HR.features[-1]["rmssd"]
    #current_feature = HR.features[-1]["LF/HF ratio"]
    
    if current_trend > 0:
        osc_client.send_message("/actuator/" + act_id + "/inflate", 100.0)
        #osc_client.send_message("/actuator/" + act_id + "/inflate", current_trend * 1000/current_feature)
        timer_exhale = threading.Timer(timer_interval, heartrate_actuation, [act_id, HR, timer_interval])
        timer_exhale.start()

    else:
        #osc_client.send_message("/actuator/inflate", current_trend * 1000/current_feature)
        osc_client.send_message("/actuator/" + act_id + "/inflate", -100.0)
        timer_inhale = threading.Timer(timer_interval, heartrate_actuation, [act_id, HR, timer_interval])
        timer_inhale.start()

#### Set up configuration: IP (BITalino, R-IoT or Node-RED), port and OSC address pattern

In [19]:
# Definitions
bitalino_ip = '192.168.0.100'
bitalino_port = 8000

riot_ip = "192.168.0.102"
riot_port = 31000 

# riot_ip = "192.168.1.40"
# riot_port = 12000 

#actuator_ip = '192.168.0.110'
actuator_ip = '192.168.0.100'
actuator_port = 32000

riot_address = "/raw/"
bitalino_address = "/0/bitalino"


# Configuration 
config_address = bitalino_address
config_ip = bitalino_ip
config_port = bitalino_port

osc_client = SimpleUDPClient(actuator_ip, actuator_port) 

actuation_flag = True

ecg_data = []
resp_data = []

B = biofeatures.breathing(data = np.ones(10), buffer_length=2000, srate=100)
HRV = biofeatures.hrv(data = np.ones(10), buffer_length=2000, srate=100)

my_dispatcher = dispatcher.Dispatcher()
my_dispatcher.map(config_address, data_handler, B, HRV)

server = osc_server.ThreadingOSCUDPServer((config_ip, config_port), my_dispatcher)
print("Serving on {}".format(server.server_address))

# Warmup
warmup_t = 10
timer_warmup = threading.Timer(warmup_t, warmup, [B,resp_data,HRV,ecg_data])
timer_warmup.start()

try: 
    server.serve_forever()
except KeyboardInterrupt:
    B.update_data_flag = False
    HRV.update_data_flag = False
    actuation_flag = False
    server.server_close()
    osc_client.send_message("/actuator/1/inflate", 0.0)
    osc_client.send_message("/actuator/2/inflate", 0.0)
except:
    raise

Serving on ('192.168.0.100', 8000)
Breathing WARMUP
{'breath_avg_len': 1.97, 'inhale_duration': 5.12, 'avg_inhale': 1.28, 'exhale_duration': 4.73, 'avg_exhale': 1.18, 'inhale_exhale_ratio': 1.08}
ECG WARMUP
[{'nni_mean': -24.54, 'hr_mean': 164.45, 'hr_std': 64.49, 'rmssd': 2065.32, 'lf': 52477.69, 'hf': 10818.28, 'LF/HF ratio': 4.85}]
inhale:  6
inhale:  3
{'breath_avg_len': 2.11, 'inhale_duration': 5.74, 'avg_inhale': 1.44, 'exhale_duration': 4.79, 'avg_exhale': 1.2, 'inhale_exhale_ratio': 1.2}
[{'nni_mean': -24.54, 'hr_mean': 164.45, 'hr_std': 64.49, 'rmssd': 2065.32, 'lf': 52477.69, 'hf': 10818.28, 'LF/HF ratio': 4.85}, {'nni_mean': -32.76, 'hr_mean': 163.22, 'hr_std': 63.4, 'rmssd': 2159.98, 'lf': 54409.35, 'hf': 11321.55, 'LF/HF ratio': 4.81}]
{'breath_avg_len': 2.2, 'inhale_duration': 6.17, 'avg_inhale': 1.54, 'exhale_duration': 4.84, 'avg_exhale': 1.21, 'inhale_exhale_ratio': 1.27}
[{'nni_mean': -24.54, 'hr_mean': 164.45, 'hr_std': 64.49, 'rmssd': 2065.32, 'lf': 52477.69, 'hf': 

{'breath_avg_len': 2.37, 'inhale_duration': 8.75, 'avg_inhale': 1.46, 'exhale_duration': 7.86, 'avg_exhale': 1.31, 'inhale_exhale_ratio': 1.11}
[{'nni_mean': -7.4, 'hr_mean': 159.34, 'hr_std': 58.95, 'rmssd': 2481.77, 'lf': 40210.76, 'hf': 16240.59, 'LF/HF ratio': 2.48}, {'nni_mean': 395.83, 'hr_mean': 169.85, 'hr_std': 56.2, 'rmssd': 216.6, 'lf': 1204.83, 'hf': 5900.78, 'LF/HF ratio': 0.2}, {'nni_mean': 390.81, 'hr_mean': 172.99, 'hr_std': 58.6, 'rmssd': 213.6, 'lf': 1292.85, 'hf': 6009.7, 'LF/HF ratio': 0.22}, {'nni_mean': -21.51, 'hr_mean': 168.54, 'hr_std': 63.69, 'rmssd': 2603.78, 'lf': 44610.9, 'hf': 17849.43, 'LF/HF ratio': 2.5}, {'nni_mean': -10.71, 'hr_mean': 166.25, 'hr_std': 63.03, 'rmssd': 2634.38, 'lf': 39806.74, 'hf': 18054.69, 'LF/HF ratio': 2.2}]
{'breath_avg_len': 2.43, 'inhale_duration': 9.17, 'avg_inhale': 1.53, 'exhale_duration': 7.87, 'avg_exhale': 1.31, 'inhale_exhale_ratio': 1.17}
[{'nni_mean': 395.83, 'hr_mean': 169.85, 'hr_std': 56.2, 'rmssd': 216.6, 'lf': 1204

{'breath_avg_len': 2.85, 'inhale_duration': 10.45, 'avg_inhale': 1.74, 'exhale_duration': 9.53, 'avg_exhale': 1.59, 'inhale_exhale_ratio': 1.1}
[{'nni_mean': -12.63, 'hr_mean': 166.28, 'hr_std': 61.65, 'rmssd': 2898.24, 'lf': 33209.74, 'hf': 19673.38, 'LF/HF ratio': 1.69}, {'nni_mean': 398.96, 'hr_mean': 169.18, 'hr_std': 57.47, 'rmssd': 209.05, 'lf': 1631.65, 'hf': 6143.28, 'LF/HF ratio': 0.27}, {'nni_mean': -24.45, 'hr_mean': 165.42, 'hr_std': 63.27, 'rmssd': 2932.69, 'lf': 11601.57, 'hf': 14805.61, 'LF/HF ratio': 0.78}, {'nni_mean': 459.02, 'hr_mean': 157.96, 'hr_std': 65.99, 'rmssd': 341.42, 'lf': 1693.11, 'hf': 10817.08, 'LF/HF ratio': 0.16}, {'nni_mean': 405.96, 'hr_mean': 169.39, 'hr_std': 59.29, 'rmssd': 236.54, 'lf': 1577.36, 'hf': 6211.51, 'LF/HF ratio': 0.25}]
inhale:  6
inhale:  3
{'breath_avg_len': 4.0, 'inhale_duration': 10.17, 'avg_inhale': 2.54, 'exhale_duration': 9.81, 'avg_exhale': 2.45, 'inhale_exhale_ratio': 1.04}
[{'nni_mean': 398.96, 'hr_mean': 169.18, 'hr_std': 5

{'breath_avg_len': 2.85, 'inhale_duration': 9.95, 'avg_inhale': 1.66, 'exhale_duration': 10.03, 'avg_exhale': 1.67, 'inhale_exhale_ratio': 0.99}
[{'nni_mean': -14.87, 'hr_mean': 159.16, 'hr_std': 64.46, 'rmssd': 2997.65, 'lf': 22063.72, 'hf': 28050.3, 'LF/HF ratio': 0.79}, {'nni_mean': -13.02, 'hr_mean': 160.63, 'hr_std': 64.62, 'rmssd': 2968.57, 'lf': 12772.15, 'hf': 23813.39, 'LF/HF ratio': 0.54}, {'nni_mean': 432.0, 'hr_mean': 160.33, 'hr_std': 58.94, 'rmssd': 243.61, 'lf': 7214.3, 'hf': 20428.54, 'LF/HF ratio': 0.35}, {'nni_mean': 426.3, 'hr_mean': 162.81, 'hr_std': 59.9, 'rmssd': 244.33, 'lf': 7811.25, 'hf': 24883.56, 'LF/HF ratio': 0.31}, {'nni_mean': 451.19, 'hr_mean': 158.86, 'hr_std': 63.61, 'rmssd': 307.95, 'lf': 5240.84, 'hf': 30988.29, 'LF/HF ratio': 0.17}]
{'breath_avg_len': 2.85, 'inhale_duration': 10.2, 'avg_inhale': 1.7, 'exhale_duration': 9.78, 'avg_exhale': 1.63, 'inhale_exhale_ratio': 1.04}
exhale:  6
[{'nni_mean': -13.02, 'hr_mean': 160.63, 'hr_std': 64.62, 'rmssd':

{'breath_avg_len': 3.33, 'inhale_duration': 9.96, 'avg_inhale': 1.99, 'exhale_duration': 10.02, 'avg_exhale': 2.0, 'inhale_exhale_ratio': 0.99}
[{'nni_mean': 477.75, 'hr_mean': 154.28, 'hr_std': 70.18, 'rmssd': 301.64, 'lf': 6083.18, 'hf': 44282.43, 'LF/HF ratio': 0.14}, {'nni_mean': 496.05, 'hr_mean': 149.82, 'hr_std': 67.39, 'rmssd': 343.37, 'lf': 7111.99, 'hf': 42399.26, 'LF/HF ratio': 0.17}, {'nni_mean': -23.0, 'hr_mean': 157.41, 'hr_std': 69.82, 'rmssd': 3125.22, 'lf': 52195.29, 'hf': 58146.6, 'LF/HF ratio': 0.9}, {'nni_mean': 475.5, 'hr_mean': 155.59, 'hr_std': 70.07, 'rmssd': 312.12, 'lf': 5450.49, 'hf': 39363.68, 'LF/HF ratio': 0.14}, {'nni_mean': 490.26, 'hr_mean': 149.96, 'hr_std': 67.06, 'rmssd': 317.01, 'lf': 4931.36, 'hf': 33918.77, 'LF/HF ratio': 0.15}]
{'breath_avg_len': 3.33, 'inhale_duration': 10.33, 'avg_inhale': 2.07, 'exhale_duration': 9.65, 'avg_exhale': 1.93, 'inhale_exhale_ratio': 1.07}
[{'nni_mean': 496.05, 'hr_mean': 149.82, 'hr_std': 67.39, 'rmssd': 343.37, 'l

...

exhale:  3
{'breath_avg_len': 2.5, 'inhale_duration': 11.08, 'avg_inhale': 1.58, 'exhale_duration': 8.9, 'avg_exhale': 1.27, 'inhale_exhale_ratio': 1.24}
[{'nni_mean': -21.47, 'hr_mean': 166.44, 'hr_std': 57.43, 'rmssd': 2854.02, 'lf': 30307.86, 'hf': 16701.58, 'LF/HF ratio': 1.81}, {'nni_mean': 388.54, 'hr_mean': 172.72, 'hr_std': 55.76, 'rmssd': 198.83, 'lf': 471.24, 'hf': 1422.84, 'LF/HF ratio': 0.33}, {'nni_mean': 394.13, 'hr_mean': 169.12, 'hr_std': 53.37, 'rmssd': 189.55, 'lf': 340.39, 'hf': 1556.51, 'LF/HF ratio': 0.22}, {'nni_mean': -13.98, 'hr_mean': 154.92, 'hr_std': 57.92, 'rmssd': 2980.69, 'lf': 23716.04, 'hf': 12593.89, 'LF/HF ratio': 1.88}, {'nni_mean': 461.03, 'hr_mean': 147.22, 'hr_std': 51.4, 'rmssd': 250.17, 'lf': 2947.17, 'hf': 9594.04, 'LF/HF ratio': 0.31}]
{'breath_avg_len': 3.33, 'inhale_duration': 9.33, 'avg_inhale': 1.56, 'exhale_duration': 10.65, 'avg_exhale': 2.13, 'inhale_exhale_ratio': 0.88}
[{'nni_mean': 388.54, 'hr_mean': 172.72, 'hr_std': 55.76, 'rmssd': 

{'breath_avg_len': 3.33, 'inhale_duration': 9.83, 'avg_inhale': 1.97, 'exhale_duration': 10.15, 'avg_exhale': 1.69, 'inhale_exhale_ratio': 0.97}
[{'nni_mean': 453.95, 'hr_mean': 156.44, 'hr_std': 63.15, 'rmssd': 287.0, 'lf': 4602.04, 'hf': 7069.83, 'LF/HF ratio': 0.65}, {'nni_mean': 442.93, 'hr_mean': 159.79, 'hr_std': 62.7, 'rmssd': 283.56, 'lf': 6271.8, 'hf': 5370.33, 'LF/HF ratio': 1.17}, {'nni_mean': 423.33, 'hr_mean': 162.03, 'hr_std': 58.34, 'rmssd': 247.87, 'lf': 5016.97, 'hf': 5220.82, 'LF/HF ratio': 0.96}, {'nni_mean': 423.11, 'hr_mean': 162.08, 'hr_std': 58.3, 'rmssd': 247.64, 'lf': 5013.36, 'hf': 5220.75, 'LF/HF ratio': 0.96}, {'nni_mean': -8.06, 'hr_mean': 159.35, 'hr_std': 61.85, 'rmssd': 2936.71, 'lf': 24819.55, 'hf': 15836.29, 'LF/HF ratio': 1.57}]
exhale:  3
{'breath_avg_len': 2.85, 'inhale_duration': 9.84, 'avg_inhale': 1.64, 'exhale_duration': 10.14, 'avg_exhale': 1.69, 'inhale_exhale_ratio': 0.97}
[{'nni_mean': 442.93, 'hr_mean': 159.79, 'hr_std': 62.7, 'rmssd': 283.

{'breath_avg_len': 2.85, 'inhale_duration': 9.85, 'avg_inhale': 1.64, 'exhale_duration': 10.13, 'avg_exhale': 1.69, 'inhale_exhale_ratio': 0.97}
[{'nni_mean': -17.21, 'hr_mean': 165.61, 'hr_std': 63.91, 'rmssd': 2913.95, 'lf': 31239.46, 'hf': 19920.39, 'LF/HF ratio': 1.57}, {'nni_mean': 430.67, 'hr_mean': 161.79, 'hr_std': 61.0, 'rmssd': 243.19, 'lf': 5836.6, 'hf': 10122.39, 'LF/HF ratio': 0.58}, {'nni_mean': 420.0, 'hr_mean': 163.43, 'hr_std': 59.34, 'rmssd': 226.24, 'lf': 5935.53, 'hf': 10018.74, 'LF/HF ratio': 0.59}, {'nni_mean': 412.55, 'hr_mean': 165.96, 'hr_std': 59.02, 'rmssd': 225.28, 'lf': 5796.29, 'hf': 9581.57, 'LF/HF ratio': 0.6}, {'nni_mean': 422.0, 'hr_mean': 162.48, 'hr_std': 59.21, 'rmssd': 221.65, 'lf': 5232.69, 'hf': 9616.63, 'LF/HF ratio': 0.54}]
{'breath_avg_len': 2.85, 'inhale_duration': 9.49, 'avg_inhale': 1.58, 'exhale_duration': 10.49, 'avg_exhale': 1.75, 'inhale_exhale_ratio': 0.9}
[{'nni_mean': 430.67, 'hr_mean': 161.79, 'hr_std': 61.0, 'rmssd': 243.19, 'lf': 

{'breath_avg_len': 2.5, 'inhale_duration': 9.49, 'avg_inhale': 1.36, 'exhale_duration': 10.49, 'avg_exhale': 1.5, 'inhale_exhale_ratio': 0.9}
[{'nni_mean': 425.0, 'hr_mean': 162.33, 'hr_std': 59.62, 'rmssd': 233.54, 'lf': 2756.26, 'hf': 6988.34, 'LF/HF ratio': 0.39}, {'nni_mean': -10.2, 'hr_mean': 157.94, 'hr_std': 64.39, 'rmssd': 3012.91, 'lf': 20901.68, 'hf': 17486.33, 'LF/HF ratio': 1.2}, {'nni_mean': 429.56, 'hr_mean': 162.68, 'hr_std': 62.21, 'rmssd': 241.66, 'lf': 2649.89, 'hf': 6032.46, 'LF/HF ratio': 0.44}, {'nni_mean': 431.59, 'hr_mean': 162.37, 'hr_std': 62.89, 'rmssd': 247.95, 'lf': 2907.69, 'hf': 5660.98, 'LF/HF ratio': 0.51}, {'nni_mean': 437.73, 'hr_mean': 162.4, 'hr_std': 65.62, 'rmssd': 246.05, 'lf': 2677.83, 'hf': 5050.73, 'LF/HF ratio': 0.53}]
{'breath_avg_len': 4.0, 'inhale_duration': 9.34, 'avg_inhale': 2.34, 'exhale_duration': 10.64, 'avg_exhale': 2.13, 'inhale_exhale_ratio': 0.88}
[{'nni_mean': -10.2, 'hr_mean': 157.94, 'hr_std': 64.39, 'rmssd': 3012.91, 'lf': 209

{'breath_avg_len': 2.85, 'inhale_duration': 10.85, 'avg_inhale': 1.81, 'exhale_duration': 9.13, 'avg_exhale': 1.52, 'inhale_exhale_ratio': 1.19}
[{'nni_mean': -14.23, 'hr_mean': 162.7, 'hr_std': 62.91, 'rmssd': 2932.92, 'lf': 32536.02, 'hf': 17871.75, 'LF/HF ratio': 1.82}, {'nni_mean': 410.65, 'hr_mean': 165.54, 'hr_std': 58.99, 'rmssd': 215.48, 'lf': 2359.39, 'hf': 4470.37, 'LF/HF ratio': 0.53}, {'nni_mean': 407.92, 'hr_mean': 165.5, 'hr_std': 56.6, 'rmssd': 213.89, 'lf': 2632.34, 'hf': 4791.7, 'LF/HF ratio': 0.55}, {'nni_mean': -12.74, 'hr_mean': 160.34, 'hr_std': 59.3, 'rmssd': 2910.26, 'lf': 23971.08, 'hf': 13724.87, 'LF/HF ratio': 1.75}, {'nni_mean': 413.11, 'hr_mean': 162.2, 'hr_std': 54.05, 'rmssd': 206.62, 'lf': 3040.71, 'hf': 5337.32, 'LF/HF ratio': 0.57}]
{'breath_avg_len': 2.85, 'inhale_duration': 10.06, 'avg_inhale': 1.68, 'exhale_duration': 9.92, 'avg_exhale': 1.42, 'inhale_exhale_ratio': 1.01}
[{'nni_mean': 410.65, 'hr_mean': 165.54, 'hr_std': 58.99, 'rmssd': 215.48, 'lf'

In [11]:
from pythonosc import dispatcher, osc_server
from pythonosc.udp_client import SimpleUDPClient
import biofeatures
import time

actuator_ip = '192.168.0.100'
actuator_port = 32000

osc_client = SimpleUDPClient(actuator_ip, actuator_port) 
osc_client.send_message("/actuator/1/inflate", 0.0)
osc_client.send_message("/actuator/2/inflate", 0.0)



In [18]:
deflate("1", 0)
deflate("2", 2)

In [ ]:
warmup_inflate("1", 3)
warmup_inflate("2", 3)

In [ ]:
actuation_flag = True

pulsating("1", True)

In [ ]:
actuation_flag = False

In [ ]:
osc_client.send_message("/actuator/1/inflate", 0.0)

In [ ]:
print("hi")

In [ ]:
deflate("2", 2)



In [ ]:
deflate("1", 2)



In [22]:
B.features

{'breath_avg_len': 2.85,
 'inhale_duration': 10.58,
 'avg_inhale': 1.76,
 'exhale_duration': 9.4,
 'avg_exhale': 1.57,
 'inhale_exhale_ratio': 1.13}